## MNIST tutorial

Original Source code: https://github.com/pytorch/examples


In [8]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
ㅕ
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct,
                len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))


### Main code

In [9]:
use_cuda = torch.cuda.is_available()

batch_size = 64
test_batch_size = 1000
lr = 0.01
momentum = 0.5
log_interval = 10
epochs = 10

device = torch.device("cuda:0" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
        batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.332243
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.294138
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.298570
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.311899
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.288042
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.277309
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.225912
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.195363
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.174541
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.169730
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.184361
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.131670
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.025631
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.022050
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.891217
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.804284
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.741564
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.512723
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.508251
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.425389
Train Epoch: 1 [12800/60000 (

/opt/conda/envs/pytorch-py3.5.5/lib/python3.5/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1774, Accuracy: 9477/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.416379
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.257549
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.471074
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.443420
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.695574
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.338557
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.639003
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.337310
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.478635
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.484244
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.455504
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.362231
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.403348
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.381127
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.235499
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.252906
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.413659
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.343517
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.255921
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.124196
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.247663
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.226866
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.212742
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.262055
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.393341
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.406158
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.195961
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.157479
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.390183
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.353361
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.236857
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.173911
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.470618
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.452410
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.320125
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.383459
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.172287
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.218936
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.210851


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.125329
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.163414
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.188242
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.109673
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.117606
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.133215
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.287298
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.146523
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.545575
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.207139
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.345051
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.225751
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.422107
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.147841
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.162038
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.150130
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.211688
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.215814
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.157218
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.081590


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.314211
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.116645
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.174704
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.237380
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.346375
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.225778
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.189283
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.253946
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.209052
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.240381
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.162622
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.203421
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.180728
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.145175
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.207482
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.162875
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.119060
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.142048
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.108159
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.049429


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.224378
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.143456
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.269001
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.212485
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.157504

Test set: Average loss: 0.0557, Accuracy: 9817/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.081445
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.141190
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.152643
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.149715
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.450524
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.217061
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.285121
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.158756
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.275876
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.207941
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.238099
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.246440
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.092947
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.162765
Train Epoch: 9 [

Train Epoch: 10 [40320/60000 (67%)]	Loss: 0.101858
Train Epoch: 10 [40960/60000 (68%)]	Loss: 0.266941
Train Epoch: 10 [41600/60000 (69%)]	Loss: 0.086986
Train Epoch: 10 [42240/60000 (70%)]	Loss: 0.183164
Train Epoch: 10 [42880/60000 (71%)]	Loss: 0.117566
Train Epoch: 10 [43520/60000 (72%)]	Loss: 0.074504
Train Epoch: 10 [44160/60000 (74%)]	Loss: 0.132849
Train Epoch: 10 [44800/60000 (75%)]	Loss: 0.304247
Train Epoch: 10 [45440/60000 (76%)]	Loss: 0.122180
Train Epoch: 10 [46080/60000 (77%)]	Loss: 0.079933
Train Epoch: 10 [46720/60000 (78%)]	Loss: 0.166000
Train Epoch: 10 [47360/60000 (79%)]	Loss: 0.076918
Train Epoch: 10 [48000/60000 (80%)]	Loss: 0.123950
Train Epoch: 10 [48640/60000 (81%)]	Loss: 0.107719
Train Epoch: 10 [49280/60000 (82%)]	Loss: 0.062868
Train Epoch: 10 [49920/60000 (83%)]	Loss: 0.174146
Train Epoch: 10 [50560/60000 (84%)]	Loss: 0.303861
Train Epoch: 10 [51200/60000 (85%)]	Loss: 0.117352
Train Epoch: 10 [51840/60000 (86%)]	Loss: 0.241473
Train Epoch: 10 [52480/60000 (8

In [14]:
torch.save(model, 'test_mnist.pth')
model2 = torch.load('test_mnist.pth')
test(model2, device, test_loader)

/opt/conda/envs/pytorch-py3.5.5/lib/python3.5/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/envs/pytorch-py3.5.5/lib/python3.5/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0483, Accuracy: 9837/10000 (98%)

